## 使用 Selector 实现意图路由

智能体路由可以扩展为更广泛的场景，比如根据不同的工具集合、向量库类型等，配置为不同的智能体，
然后根据用户的意图和场景，使用 Selector 动态选择最佳的智能体。


Selector 有两个参数：
- condition 可以自定义，或者采用内置的方法名称，包括：
    - first : 第一个，这是默认选项，主要用于测试
    - random : 随机选一个
    - similar : 输入与备选 Runnable 的 description 属性接近（要使用这一方法必须提供向量嵌入模型）
- agents 智能体列表

### 基础示范

In [1]:
from illufly.chat import ChatQwen, ChatZhipu, FakeLLM
from illufly.flow import Selector
from illufly.io import alog

# 默认选择了第一个 Runnable 作为 selected
agent = Selector("first", [FakeLLM(["abc", "df"]), ChatQwen(), ChatZhipu()])
agent.select()
agent.bind_provider({"task": "写一首歌"})

agent("你是什么模型？", verbose=True)

[INFO] 记住 10 轮对话
[INFO] I am FakeLLM
abc


'abc'

In [2]:
agent.selected("你是什么模型？", verbose=True)

[INFO] 记住 10 轮对话
[INFO] I am FakeLLM
df


'df'

In [3]:
agent.selected

<FakeLLM.4869701584>

### 路由 ChatAgent

考虑到你有两个专业的 ChatAgent:

- data_agent 擅长数据分析
- writer_agent 擅长写作

你已经根据为他们分别配置了提示语模板、工具集、数据库等，但现在你需要根据用户的意图匹配到底使用哪一个智能体。<br>

那么如何判断用户意图呢？
- 方案1 可以使用大模型来做甄别，这会增加一个环节，推理过程可能延长用户的等待
- 方案2 可以根据问题做文本相似性比较，这样做效率很高

我们的例子中探讨方案2如何实现，关键就是与什么做比较？

原始的可比较信息是：用户问题与智能体描述文本之间的比较，但这明显不太够用。<br>
可以使用**检索器**和**经验缓存**来弥补。


In [4]:
from illufly.chat import ChatQwen, ChatZhipu, FakeLLM
from illufly.flow import Selector
from illufly.rag import TextEmbeddings

agent = Selector(
    condition="similar",
    runnables=[
        FakeLLM(description="模拟调用"),
        ChatQwen(description="写歌"),
        ChatZhipu(description="数据分析")
    ],
    embeddings=TextEmbeddings()
)

task = "写一首歌，3句"
agent.bind_provider({"task": task})
agent(task)

月光洒在我的脸上，  
心中涌动着无尽的希望，  
夜色中，我听见梦想在歌唱。


'月光洒在我的脸上，  \n心中涌动着无尽的希望，  \n夜色中，我听见梦想在歌唱。'

In [5]:
agent.consumer_dict

{'task': '写一首歌，3句'}

### 路由 PromptTemplate

In [6]:
from illufly.types import PromptTemplate
from illufly.flow import Selector

r = Selector(
    runnables=[PromptTemplate(text="请帮我：{{task}}"), PromptTemplate(text="你是一个作家")]
)
r.bind_provider({"task": "写一首歌"})

r.selected

使用 lazy_binding_map 可以先声明对象，然后在合适的时机自动实现绑定。

In [1]:
from illufly.chat import ChatQwen
from illufly.flow import Selector
from illufly.types import PromptTemplate

t = Selector(
    condition=lambda vars, runs: runs[0],
    runnables=[
        PromptTemplate(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        PromptTemplate(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ]
)
qwen = ChatQwen(memory=t)
qwen("小兔子")

小兔子白又白，  
两耳竖起来，  
爱吃萝卜和青菜，  
蹦蹦跳跳真可爱。


'小兔子白又白，  \n两耳竖起来，  \n爱吃萝卜和青菜，  \n蹦蹦跳跳真可爱。'

## 自定义 Selector

### 返回 Runnable

In [3]:
from illufly.chat import ChatQwen
from illufly.flow import Selector
from illufly.types import PromptTemplate

t = Selector(
    runnables=[
        PromptTemplate(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        PromptTemplate(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ],
    condition=lambda vars, runs: runs[0]
)
t.select()

<PromptTemplate consumer_dict={'task'} text='请帮我写一首关于《{{task}}》的儿歌，四句'>

### 返回 name

In [4]:
from illufly.chat import ChatQwen
from illufly.flow import Selector
from illufly.types import PromptTemplate

t = Selector(
    runnables=[
        PromptTemplate(text="请帮我写一首关于《{{task}}》的儿歌，四句"),
        PromptTemplate(text="请帮我起一个{{name}}的名字，3个字", lazy_binding_map={"name": "task"})
    ],
    condition=lambda vars, runs: runs[0].name
)
t.select()

'PromptTemplate.4355606272'

### 返回 End

In [8]:
from illufly.flow import Selector

t = Selector(
    condition=lambda: "__End__"
)
t.selected

'__End__'

## select 方法和 selected 属性

由于 `Selector` 的路由规则不是幂等操作，因此提供了专门的 `select` 方法来实现路由操作，而 `selected` 属性仅仅是取得路由后的结果。因此，会发生以下行为：

- 如果 Selector 从未执行过 select 方法就要求提取 selected 属性，则自动执行一次 select 方法
- 如果不重新执行 select 则 selected 属性始终返回上一次 select 结果